In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np
import pandas as pd

from keras.utils import to_categorical
import os
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

!pip install scikeras[tensorflow]

from scikeras.wrappers import KerasClassifier

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Function to read data from CSV files
def read_data_from_csv(folder_path):
    dfs = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".csv"):
            file_path = os.path.join(folder_path, filename)
            df = pd.read_csv(file_path)
            dfs.append(df)
    return pd.concat(dfs, ignore_index=True)

In [ ]:
# Read data from train and test folders
combined_data = read_data_from_csv('/content/drive/My Drive/datasets/test')


In [ ]:
# Encoding categorical variables
encoder = LabelEncoder()
combined_data['label'] = encoder.fit_transform(combined_data['label'])


In [ ]:
X_train_combined, X_test_combined, y_train_combined, y_test_combined = train_test_split(
    combined_data.drop(columns=['label']),  # Features
    combined_data['label'],  # Target variable
    test_size=0.2,  # 20% of data will be used for testing
    random_state=42,  # For reproducibility
    stratify=combined_data['label']  # Ensure same label proportions in train and test sets
)

In [ ]:
# Scaling numerical variables
scaler = StandardScaler()
X_train_combined_scaled = scaler.fit_transform(X_train_combined)
X_test_combined_scaled = scaler.transform(X_test_combined)

In [ ]:
# Define a function to create the Keras model
def create_keras_model():
    model = Sequential([
        Dense(64, input_dim=X_train_combined_scaled.shape[1], activation='relu'),
        Dense(32, activation='relu'),
        Dense(len(np.unique(y_train_combined)), activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Wrap the Keras model in a scikit-learn compatible estimator
keras_classifier = KerasClassifier(build_fn=create_keras_model, epochs=10, batch_size=32, verbose=0)

# Define base learners
base_learners = [
    ('random_forest', RandomForestClassifier(n_estimators=100)),
    ('dnn', keras_classifier),
    ('knn', KNeighborsClassifier())
]

# Define meta-learner
meta_learner = LogisticRegression()

In [14]:


# Create stacking classifier
stacking_classifier = StackingClassifier(estimators=base_learners, final_estimator=meta_learner)

# Train stacking classifier
stacking_classifier.fit(X_train_combined_scaled, y_train_combined)


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/scikeras

StackingClassifier(estimators=[('random_forest', RandomForestClassifier()),
                               ('dnn',
                                KerasClassifier(batch_size=32, build_fn=<function create_keras_model at 0x7d973a07eb90>, epochs=10, verbose=0)),
                               ('knn', KNeighborsClassifier())],
                   final_estimator=LogisticRegression())

In [ ]:
# Make predictions
stacking_predictions = stacking_classifier.predict(X_test_combined)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [ ]:
# Calculate accuracy
stacking_accuracy = accuracy_score(y_test_combined, stacking_predictions)

# Print accuracy
print("Stacking Accuracy:", stacking_accuracy)

In [ ]:
# Print classification report
print("Classification Report for Stacking:")
print(classification_report(y_test_combined, stacking_predictions))